### Import knihoven a konfigurace

In [ ]:
import argparse
import os
import shutil
import re

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2

from pypylon import pylon 
from pypylon_opencv_viewer import BaslerOpenCVViewer

from IPython.display import clear_output, display

import pytesseract
from PIL import Image

from collections import OrderedDict
import math

import yaml
from natsort import natsorted
from scipy.spatial import distance

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

%matplotlib inline

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

<a id='connect_functions'>Funkce k připojení kamery podle sériového čísla.</a> Umožňuje poté tlačítkem zapnout snímání aktuální scény. V případě nutnosti změny nastavení připojení kamery, je třeba restartovat jupyter notebook tlačítkem `restart kernel (with dialog)`. Kamera se bude tvářit jako, že je již připojená k jinému zařízení. (`RuntimeException: Failed to open 'Basler ...'. The device is controlled by another application.`)

In [ ]:
# Configuration PyPylon Viewer to load features for RGB Matrix Camera
VIEWER_CONFIG_RGB_MATRIX = {
    "features": [
        {
            "name": "GainRaw",
            "type": "int",
            "step": 1,
        },
        {
            "name": "Height",
            "type": "int",
            "unit": "px",
            "step": 2,
        },
        {
            "name": "Width",
            "type": "int",
            "unit": "px",
            "step": 2,
        },
        {
            "name": "CenterX",
            "type": "bool",
        },
        {
            "name": "CenterY",
            "type": "bool",

        },
        {
            "name": "OffsetX",
            "type": "int",
            "dependency": {"CenterX": False},
            "unit": "px",
            "step": 2,
        },
        {
            "name": "OffsetY",
            "type": "int",
            "dependency": {"CenterY": False},
            "unit": "px",
            "step": 2,
        },
        {
            "name": "AcquisitionFrameRateAbs",
            "type": "int",
            "unit": "fps",
            "dependency": {"AcquisitionFrameRateEnable": True},
            "value": 30,
            "max": 150,
            "min": 1,
        },
        {
            "name": "AcquisitionFrameRateEnable",
            "type": "bool",
        },
        {
            "name": "ExposureAuto",
            "type": "choice_text",
            "options": ["Off", "Once", "Continuous"],
            "style": {"button_width": "90px"}
        },
        {
            "name": "ExposureTimeAbs",
            "type": "int",
            "dependency": {"ExposureAuto": "Off"},
            "unit": "μs",
            "step": 100,
            "max": 35000,
            "min": 500,
        },
        {
            "name": "BalanceWhiteAuto",
            "type": "choice_text",
            "options": ["Off", "Once", "Continuous"],
            "style": {"button_width": "90px"}
        },
    ],
    "features_layout": [
        ("Height", "Width"), 
        ("OffsetX", "CenterX"), 
        ("OffsetY", "CenterY"), 
        ("ExposureAuto", "ExposureTimeAbs"),
        ("AcquisitionFrameRateAbs", "AcquisitionFrameRateEnable"),
        ("BalanceWhiteAuto", "GainRaw")
    ],
    "actions_layout": [
        ("StatusLabel"),
        ("SaveConfig", "LoadConfig", "ContinuousShot", "SingleShot"), 
        ("UserSet")
    ],
    "default_user_set": "UserSet3",
}

In [ ]:
# Configuration PyPylon Viewer to load features for Monochromatic Matrix Camera
VIEWER_CONFIG_MONO_MATRIX = {
    "features": [
         {
            "name": "GainRaw",
            "type": "int",
            "step": 1,
        },
        {
            "name": "Height",
            "type": "int",
            "unit": "px",
            "step": 2,
        },
        {
            "name": "Width",
            "type": "int",
            "unit": "px",
            "step": 2,
        },
        {
            "name": "CenterX",
            "type": "bool",
        },
        {
            "name": "CenterY",
            "type": "bool",

        },
        {
            "name": "OffsetX",
            "type": "int",
            "dependency": {"CenterX": False},
            "unit": "px",
            "step": 2,
        },
        {
            "name": "OffsetY",
            "type": "int",
            "dependency": {"CenterY": False},
            "unit": "px",
            "step": 2,
        },
        {
            "name": "AcquisitionFrameRateAbs",
            "type": "int",
            "unit": "fps",
            "dependency": {"AcquisitionFrameRateEnable": True},
            "value": 30,
            "max": 150,
            "min": 1,
        },
        {
            "name": "AcquisitionFrameRateEnable",
            "type": "bool",
        },
        {
            "name": "ExposureAuto",
            "type": "choice_text",
            "options": ["Off", "Once", "Continuous"],
            "style": {"button_width": "90px"}
        },
        {
            "name": "ExposureTimeAbs",
            "type": "int",
            "dependency": {"ExposureAuto": "Off"},
            "unit": "μs",
            "step": 500,
            "max": 35000,
            "min": 500,
        },
    ],
    "features_layout": [
        ("Height", "Width"), 
        ("OffsetX", "CenterX"), 
        ("OffsetY", "CenterY"), 
        ("ExposureTimeAbs", "ExposureAuto"),
        ("AcquisitionFrameRateAbs", "AcquisitionFrameRateEnable"),
        ("GainRaw")
    ],
    "actions_layout": [
        ("StatusLabel"),
        ("SaveConfig", "LoadConfig", "ContinuousShot", "SingleShot"), 
        ("UserSet")
    ],
    "default_user_set": "UserSet3",
}

In [ ]:
# Configuration PyPylon Viewer to load features for Monochromatic Line Scan Camera
VIEWER_CONFIG_MONO_LINE = {
    "features": [
         {
            "name": "GainRaw",
            "type": "int",
            "step": 1,
        },
        {
            "name": "Height",
            "type": "int",
            "unit": "px",
            "step": 2,
        },
        {
            "name": "Width",
            "type": "int",
            "unit": "px",
            "step": 2,
        },
        {
            "name": "CenterX",
            "type": "bool",
        },
        {
            "name": "OffsetX",
            "type": "int",
            "dependency": {"CenterX": False},
            "unit": "px",
            "step": 2,
        },
        {
            "name": "AcquisitionLineRateAbs",
            "type": "int",
            "max": 5000,
            "min": 100,
            "step": 100,
        },
        {
            "name": "ExposureAuto",
            "type": "choice_text",
            "options": ["Off", "Once", "Continuous"],
            "style": {"button_width": "90px"}
        },
        {
            "name": "ExposureTimeAbs",
            "type": "int",
            "dependency": {"ExposureAuto": "Off"},
            "unit": "μs",
            "step": 500,
            "max": 35000,
            "min": 500,
        },
    ],
    "features_layout": [
        ("Height", "Width"), 
        ("OffsetX", "CenterX"), 
        ("ExposureTimeAbs", "ExposureAuto"),
        ("AcquisitionLineRateAbs", "GainRaw")
    ],
    "actions_layout": [
        ("StatusLabel"),
        ("SaveConfig", "LoadConfig", "ContinuousShot", "SingleShot"), 
        ("UserSet")
    ],
    "default_user_set": "UserSet3",
}

In [ ]:
# Configuration PyPylon Viewer to load features for Monochromatic Matrix Camera with pericentric lens
VIEWER_CONFIG_MONO_MATRIX_PERICENTRIC = {
    "features": [
         {
            "name": "Gain",
            "type": "int",
            "unit": "dB",
            "step": 1,
        },
        {
            "name": "Height",
            "type": "int",
            "unit": "px",
            "step": 2,
            "value": 1270
        },
        {
            "name": "Width",
            "type": "int",
            "unit": "px",
            "step": 2,
            "value": 1344
        },
        {
            "name": "OffsetX",
            "type": "int",
            "unit": "px",
            "step": 2,
            "value": 608
        },
        {
            "name": "OffsetY",
            "type": "int",
            "unit": "px",
            "step": 2,
            "value": 418
        },
        {
            "name": "ExposureTime",
            "type": "int",
            "max": 35000,
            "min": 90,
            "step": 30,
            "value": 1500,
            "unit": "μs",
        },
    ],
    "features_layout": [
        ("Height", "Width"), 
        ("OffsetX", "OffsetY"), 
        ("Gain", "ExposureTime"),
    ],
    "actions_layout": [
        ("StatusLabel"),
        ("SaveConfig", "LoadConfig", "ContinuousShot", "SingleShot"), 
        ("UserSet")
    ],
    "default_user_set": "UserSet3",
}

In [ ]:
def connect_camera(serial_number):
    ''' Connects camera specified with its serial number
    
    Parameters
    ----------
    serial_number : string
        Camera's serial number.
    grabbed_images_path : string
        Path to folder where the saved images will be stored.
    Returns
    -------
    camera : object
    '''
    info = None
    for i in pylon.TlFactory.GetInstance().EnumerateDevices():
        if i.GetSerialNumber() == serial_number:
            info = i
            break
    else:
        print('Camera with {} serial number not found'.format(serial_number))

    # VERY IMPORTANT STEP! To use Basler PyPylon OpenCV viewer you have to call .Open() method on you camera
    if info is not None:
        camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateDevice(info)) 
        camera.Open()
        return camera
    else:
        return None    

<a id='load_save_functions'>Funkce pro načtení/uložení obrázku</a>

In [ ]:
def load_image(file_path):
    assert os.path.exists(file_path), 'File does NOT exist! (' + file_path + ')'
    return cv2.imread(file_path)

def save_image(image, file_path):
    return cv2.imwrite(file_path, image)

Funkce pro zobrazení okna s přidáním eventu na klik myší.

In [ ]:
def show_camera_window(*imgs, scale=1):
    def print_xy(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONUP:
            print('x = %d, y = %d'% (x, y))  
        
    for i, img in enumerate(imgs, 1):
        window_name_id = 'Camera capture' + ' ' + str(i)
        
        h,w = img.shape[:2]
        cv2.namedWindow(window_name_id, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
        cv2.resizeWindow(window_name_id, int(w * scale), int(h * scale))
        cv2.setMouseCallback(window_name_id, print_xy)
        if len(imgs) > 1:
            cv2.moveWindow(window_name_id, (i-1)*int(w * scale), 0)
        cv2.imshow(window_name_id, img)

<a id='preprocessing_functions'>Metody předzpracování.</a>

In [ ]:
def to_gray(img_bgr):
    ''' Converts image to monochrome
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Output image.
    '''
    if len(img_bgr.shape) == 2:
        return img_bgr
    return cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

def to_hsv(img_bgr):
    ''' Converts image to HSV (hue, saturation, value) color space.
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Output image.
    '''
    dst = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    return dst

def to_rgb(img_bgr):
    ''' Converts image to RGB (red, green, blue) color space from BGR.
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Output image.
    '''
    dst = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    return dst

def negative(img):
    ''' Converts image to its negative.
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Output image.
    '''
    dst = 255 - img
    return dst

def crop(img, tl_x, tl_y, br_x, br_y):
    ''' Crops image by added coordinates.
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    tl_x : int
        TOP-LEFT corner's x-coordinate
    tl_y : int
        TOP-LEFT corner's y-coordinate
    br_x : int
        BOTTOM-RIGHT corner's x-coordinate
    br_y : int
        BOTTOM-RIGHT corner's y-coordinate
    Returns
    -------
    Output image.
    '''
    roi = img[tl_y:br_y, tl_x:br_x]
    return roi    

def crop_by_bounding_rect(img_bin):
    ''' Crops binary image by ONE bounding rectangle corresponding to ALL objects in the binary image.
    
    Parameters
    ----------
    img_bin : numpy.ndarray
        Input binary image.
    Returns
    -------
    Output cropped image.
    '''
    assert len(img_bin.shape) == 2, 'Input image is NOT binary!'
    
    contours, _  = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    tl_x, tl_y, w, h = cv2.boundingRect(contours[0])
    return crop(img_bin, tl_x, tl_y, tl_x+w, tl_y+h)

def crop_contour(contour, image):
    ''' Crops contour in respect to its bounding rectangle. 
    It's the fastest method, but could include other parts 
    of image than just contour if the contour is irregulary shaped.
    
    Parameters
    ----------
    contour : numpy.ndarray
        Contour that represents the area from image to be cropped. 
        The bounding rectangle of contour is used.
    img_bin : numpy.ndarray
        Input binary image.
    Returns
    -------
    Output cropped image.
    '''
    x,y,w,h = cv2.boundingRect(contour)
    return image[y:y+h, x:x+w]

def contour_to_image(contour, image, size=None):
    ''' Creates new image from the contour. 
    It's similar to contour cropping but it's not that fast. 
    It does not suffer from the known error if the contour is irregulary shaped.
    
    Parameters
    ----------
    contour : numpy.ndarray
        Contour that represents the area from image to be cropped. 
    img_bin : numpy.ndarray
        Input binary image.
    size : tuple
        Optional size of the created image. 
        If it's not used, the image's size is the same as the 
        size of bounding rectangle of the input contour.
    Returns
    -------
    Output cropped image.
    '''
    if size is None:
        _, _, w, h = cv2.boundingRect(contour)
        size = (w, h)

    assert type(size) is tuple, 'Param size should be a tuple!'
    blank = np.zeros_like(image)
    half_x = int(size[0] * 0.5)
    half_y = int(size[1] * 0.5)

    c = get_center(contour)
    cv2.drawContours(blank, [contour], -1, (255, 255, 255), cv2.FILLED)

    return blank[c[1]-half_y:c[1]+half_y, c[0]-half_x:c[0]+half_x].copy()

def resize(image, size, method=cv2.INTER_AREA):
    ''' Resizes the image to the preffered size.  
    Method of resizing is well suited for making the images smaller rather than larger
    (cv2.INTER_AREA). For making images larger, use other cv2.INTER_### instead.
    
    Parameters
    ----------
    image : numpy.ndarray
        Contour that represents the area from image to be cropped. 
    size : tuple
        New size of the resized image. 
    method : int
        Optional argument. For more information see cv2.INTER_### parameters.
    Returns
    -------
    Output resized image.
    '''
    assert type(size) is tuple, 'Variable size is NOT a tuple!'
    return cv2.resize(image, size, method)

In [ ]:
def rotated_rectangle(image, idx):
    ''' Draws rotated rectangle into the image from indexes of binary image. 
    You can get the indexes of objects from binary image using cv2.findNonZero().
    Input image is not modified.
    '''
    res = image.copy()
    rect = cv2.minAreaRect(idx)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(res, [box], -1, (255, 255, 255), 1)
    return res, rect

def get_center(contour):
    ''' Gets the center of contour in pixels in tuple format.
    
    Parameters
    ----------
    contour : numpy.ndarray
        input contour.
    Returns
    -------
    Center in pixels in tuple format.
    '''
    M = cv2.moments(contour)
    cX = int(M['m10'] / M['m00'])
    cY = int(M['m01'] / M['m00'])
    
    return (cX, cY)

<a id='segmentation_functions'>Metody segmentace.</a>

In [ ]:
def segmentation_one_threshold(img, threshold):
    '''Segments image into black & white using one threshold
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    threshold : int
        Pixels with value lower than threshold are considered black, the others white.
    Returns
    -------
    Output image.
    '''
    _, dst = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
    return dst

def segmentation_auto_threshold(img):
    '''Segments image into black & white using automatic threshold
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Output image.
    '''
    _, dst = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return dst

def segmentation_two_thresholds(img, lower, higher):
    '''Segments image into black & white using two thresholds
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    lower : int
        Pixels with value lower than threshold are considered black, the others white.
    higher : int
        Pixels with value higher than threshold are considered black, the others white.
    Returns
    -------
    Output image.
    '''
    return cv2.inRange(img, min(lower, higher), max(lower, higher))

def segmentation_adaptive_threshold(img, size, constant=0):
    '''Segments image into black & white using calculated adaptive 
    threshold using Gaussian function in pixel neighbourhood.
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    size : int
        Size of used gaussian. Lowest value is 3. Algorithm uses only odd numbers.
    constant : int
        Value that is added to calculated threshlod. It could be negative as well as zero as well as positive number.
    Returns
    -------
    Output binary image.
    '''
    if size < 3:
        size = 3
    elif size % 2 == 0:
        size -= 1
    return cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, size, int(constant))

def mask(img, mask_bin):
    '''Masks colored image with binary mask. Output image is just logical AND between two images.'''
    return cv2.bitwise_and(img, img, mask = mask_bin)

def find_contours(img_bin, min_area=0, max_area=1000000, fill=True):
    '''Finds contours in binary image and filters them using their area. Then it draws binary image
    from filtered contours. It counts contours as well.
    
    Parameters
    ----------
    img_bin : numpy.ndarray
        Input binary image.
    min_area : int
        Size of contour that is used to filter all smaller contours out.
    max_area : int
        Size of contour that is used to filter all larger contours out.
    Returns
    -------
    contour_drawn : numpy.ndarray
        Output binary image with drawn filled filtered contours.
    count : int
        Number of found and filtered contours.
    contours : list
        Found contours.
    '''
    contours, _  = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours =  [c for c in contours if cv2.contourArea(c) > min_area and cv2.contourArea(c) < max_area]
    thick = cv2.FILLED
    if not fill: thick = 2
    contour_drawn = cv2.drawContours(np.zeros(img_bin.shape, dtype=np.uint8), contours, -1, color=(255, 255, 255), thickness=thick)
    return contour_drawn, len(contours), contours    

<a id='filtration_functions'>Metody filtrace.</a>

In [ ]:
def filtration_box(img, filter_size):
    '''Filters image noise using box blur algorithm
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    filter_size : int
        Size of box blur filter.
    Returns
    -------
    Output image.
    '''
    return cv2.blur(img, (filter_size, filter_size))

def filtration_median(img, filter_size):
    '''Filters image noise using median algorithm
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    filter_size : int
        Size of median filter.
    Returns
    -------
    Output image.
    '''
    return cv2.medianBlur(img, filter_size)   

def filtration_gauss(img, filter_size, sigma_x):
    '''Filters image noise using Gaussian blur algorithm
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    filter_size : int
        Size of Gaussian filter.
    Returns
    -------
    Output image.
    '''
    return cv2.GaussianBlur(img, (filter_size, filter_size), sigma_x) 

def fill_holes(img_bin, close=False, size=5):
    '''Fill holes in found contours. It could merge the contour using close input with appropriate size.
    
    Parameters
    ----------
    img_bin : numpy.ndarray
        Input binary image.
    close : boolean
        If it should merge contours with missing points using close operation.
    size : int
        Size of close operation element.
    Returns
    -------
    Output binary image.
    '''
    if close:
        struct = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (size, size))
        img_bin = cv2.morphologyEx(img_bin, cv2.MORPH_CLOSE, struct)
    res, _, _ = find_contours(img_bin)
    return res

<a id='show_functions'>Metoda pro zobrazení různého množství obrázků.</a>

In [ ]:
def show_images(*imgs, scale=1, window_name='Image preview'):
    """ Opens multiple image previews depending on the length of the input *imgs list.
    The preview is terminated by pressing the 'q' key.
    
    Parameters
    ----------
    *imgs : list
        Multiple input images which have to be shown.
    scale : double
        Scale of shown image window.
    window_name : Optional[string]
        An optional window name.
    Returns
    -------
    None
    """
    def print_xy(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONUP:
            print('x = %d, y = %d'% (x, y)) 
            
    for i, img in enumerate(imgs, 1):
        h,w = img.shape[:2]
        window_name_id = window_name + ' ' + str(i)
        cv2.namedWindow(window_name_id, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
        cv2.resizeWindow(window_name_id, int(w * scale), int(h * scale))
        cv2.setMouseCallback(window_name_id, print_xy)
        cv2.moveWindow(window_name_id, (i-1)*int(w * scale), 0)

    while 1:
        for i, img in enumerate(imgs, 1):
            cv2.imshow(window_name + ' ' + str(i), img)
            
        k = cv2.waitKey(0)
        
        if k == ord('q') or k == 27:
            break

    cv2.destroyAllWindows()
    
def plot_images(*imgs):
    f = plt.figure(figsize=(30, 20))
    width = math.ceil(math.sqrt(len(imgs)))
    height = math.ceil(len(imgs) / width)
    for i, img in enumerate(imgs, 1):
        ax = f.add_subplot(height, width, i)
        ax.imshow(img)

<a id='ocr'>Metoda pro OCR obrazu</a>

In [ ]:
def ocr(img_bin):
    '''Detects text in the file.
    
    Parameters
    ----------
    img_bin : numpy.ndarray
        Input binary image. White objects on black background.
    Returns
    -------
    Text on image.
    '''
    # Tesseract works with black objects on white background.
    img_bin = negative(img_bin)
    return pytesseract.image_to_string(Image.fromarray(img_bin))

<a id='others'>Metody ostatní</a>

In [ ]:
def to_intensity(hue_angle):
    '''Converts color angle in HUE definition into intensity value of brightness image in opencv.
    
    Parameters
    ----------
    hue_angle : int
        Angle in HUE definition.
    Returns
    -------
    Integer value that represents the same HUE value but in opencv brightness image.
    '''
    return int(hue_angle * 0.5)

def to_angle(hue_intensity):
    '''Converts hue intensity value of brightness image in opencv into hue angle in HUE definition.
    
    Parameters
    ----------
    hue_intensity : int
        Intensity value of brightness image.
    Returns
    -------
    Integer value that represents the HUE angle.
    '''
    return hue_intensity * 2

def logical_and(bin_im, bin_mask):
    return cv2.bitwise_and(bin_im, bin_mask)

def order_points(pts):
    '''Sorts the points based on their x-coordinates.'''
    xSorted = pts[np.argsort(pts[:, 0]), :]

    # grab the left-most and right-most points from the sorted
    # x-roodinate points
    leftMost = xSorted[:2, :]
    rightMost = xSorted[2:, :]

    # now, sort the left-most coordinates according to their
    # y-coordinates so we can grab the top-left and bottom-left
    # points, respectively
    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (bl, tl) = leftMost

    # now that we have the top-left coordinate, use it as an
    # anchor to calculate the Euclidean distance between the
    # top-left and right-most points; by the Pythagorean
    # theorem, the point with the largest distance will be
    # our bottom-right point
    rightMost = rightMost[np.argsort(rightMost[:, 1]), :]
    (br, tr) = rightMost

    # return the coordinates in top-left, top-right,
    # bottom-right, and bottom-left order
    return np.array([tl, tr, br, bl], dtype="float32")

<a id='cart_polar_functions'>Metody transformací</a>. Například převodu mezi kartézskými a polárními souřadnicemi nebo rotace.

In [ ]:
# Linear polar warp help function
def polar_warp(img, full_radius=True, inverse=False):
    center = (img.shape[0]/2.0, img.shape[1]/2.0)
    
    if full_radius:
        radius = np.sqrt(((img.shape[0]/2.0)**2.0)+((img.shape[1]/2.0)**2.0))
    else:
        radius = center[0]
    
    method = cv2.WARP_FILL_OUTLIERS
    if inverse: 
        method += cv2.WARP_INVERSE_MAP
    dest = cv2.linearPolar(img, center, radius, method)
    return dest

In [ ]:
def warp_to_cartesian(img, full_radius=True):
    return polar_warp(img, full_radius)

def warp_to_polar(img, full_radius=True):
    return polar_warp(img, full_radius, True)

def rotate(img, angle):
    height, width = img.shape[:2]
    image_center = (width/2, height/2)

    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)

    abs_cos = abs(rotation_mat[0,0])
    abs_sin = abs(rotation_mat[0,1])

    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]

    dest = cv2.warpAffine(img, rotation_mat, (bound_w, bound_h))
    return dest

<a id='artificial_funtions'>Metody na tvorbu umělých obrázků</a>

In [ ]:
def artificial_circle_image(size):
    img_art_circ = np.ndarray((size, size), dtype=np.float32)
    step = 10
    for i in range(step, size, step):
        cv2.circle(img_art_circ, (int(size/2.0), int(size/2.0)), i-step, np.random.randint(0,255), thickness=4)
    return img_art_circ

<a id='shape_desc_funtions'>Třída pro práci s popisnými charakteristikami tvarů a jednotlivé dílčí funkce.</a>

In [ ]:
# Dimensionless descriptors
class ShapeDescriptors:
    def form_factor(area, perimeter):
        return (4 * np.pi * area) / (perimeter * perimeter)
    
    def roundness(area, max_diameter):
        return (4 * area) / (np.pi * max_diameter * max_diameter)
    
    def aspect_ratio(min_diameter, max_diameter):
        return min_diameter / max_diameter;
    
    def convexity(perimeter, convex_perimeter):
        return convex_perimeter / perimeter
    
    def solidity(area, convex_area):
        return area / convex_area
    
    def compactness(area, max_diameter):
        return np.sqrt(4 / np.pi * area) / max_diameter;
        
    def extent(area, bounding_rectangle_area):
        return area / bounding_rectangle_area;

# Špičatost
def form_factor(bin_im):
    _, _, conts = find_contours(bin_im)
    return ShapeDescriptors.form_factor(cv2.contourArea(conts[0]), cv2.arcLength(conts[0], True))

# Kulatost
def roundness(bin_im):
    _, _, conts = find_contours(bin_im)
    area = cv2.contourArea(conts[0])
    _,radius = cv2.minEnclosingCircle(conts[0])
    r = ShapeDescriptors.roundness(area, 2*radius)
    if r > 1: r = 1
    return r

# Poměr stran
def aspect_ratio(bin_im):
    _, _, conts = find_contours(bin_im)
    dims = cv2.minAreaRect(conts[0])[1]
    min_diameter = min(dims)
    max_diameter = max(dims)
    return ShapeDescriptors.aspect_ratio(min_diameter, max_diameter)
    
# Konvexita, vypouklost
def convexity(bin_im):
    _, _, conts = find_contours(bin_im)
    hull = cv2.convexHull(conts[0], None, True, True)
    per = cv2.arcLength(conts[0], True)
    conv_per = cv2.arcLength(hull, True)
    r = ShapeDescriptors.convexity(per, conv_per)
    if r > 1: r = 1
    return r 

# Plnost, celistvost
def solidity(bin_im):
    _, _, conts = find_contours(bin_im)
    hull = cv2.convexHull(conts[0], None, True, True)
    area = cv2.contourArea(conts[0])
    conv_area = cv2.contourArea(hull)
    r = ShapeDescriptors.solidity(area, conv_area)
    if r > 1: r = 1
    return r 
    
# Kompaktnost, hutnost
def compactness(bin_im):
    _, _, conts = find_contours(bin_im)
    area = cv2.contourArea(conts[0])
    max_diameter = max(cv2.minAreaRect(conts[0])[1])
    r = ShapeDescriptors.compactness(area, max_diameter)
    if r > 1: r = 1
    return r 
    
# Dosah, rozměrnost
def extent(bin_im):
    _, _, conts = find_contours(bin_im)
    area = cv2.contourArea(conts[0])
    w, h = cv2.minAreaRect(conts[0])[1]
    return ShapeDescriptors.extent(area, w*h)

<a id='measuring_funtions'>Metody pro práci při segmentaci a měření objektů</a>

In [ ]:
def copy_to(src, dst, mask):
    '''Python alternative to C++/Java OpenCV's Mat.copyTo().
    More: https://docs.opencv.org/trunk/d3/d63/classcv_1_1Mat.html#a626fe5f96d02525e2604d2ad46dd574f'''
    locs = np.where(mask != 0) # Get the non-zero mask locations
    dst[locs[0], locs[1]] = src[locs[0], locs[1]]
    return dst

def midpoint(ptA, ptB):
    '''Returns the midpoint between two input points.'''
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

def order_points(pts):
    '''Sorts the points based on their x-coordinates.'''
    xSorted = pts[np.argsort(pts[:, 0]), :]

    # grab the left-most and right-most points from the sorted
    # x-roodinate points
    leftMost = xSorted[:2, :]
    rightMost = xSorted[2:, :]

    # now, sort the left-most coordinates according to their
    # y-coordinates so we can grab the top-left and bottom-left
    # points, respectively
    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (bl, tl) = leftMost

    # now that we have the top-left coordinate, use it as an
    # anchor to calculate the Euclidean distance between the
    # top-left and right-most points; by the Pythagorean
    # theorem, the point with the largest distance will be
    # our bottom-right point
    rightMost = rightMost[np.argsort(rightMost[:, 1]), :]
    (br, tr) = rightMost

    # return the coordinates in top-left, top-right,
    # bottom-right, and bottom-left order
    return np.array([tl, tr, br, bl], dtype="float32")

def rotate_image(image, angle, image_center=None):
    """ Rotates the input image by specified angle.
    
    Parameters
    ----------
    image : np.ndarray
        Image to be rotated.
    angle : float
        Rotation angle.
    image_center : Optional[tuple(int, int)]
        Center of rotation.
    Returns
    -------
    np.ndarray
        Returns the rotated input image by specified angle.
    """
    if image_center is None:
        image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

# TODO: Most inefficient function in my whole life 
def draw_rotated_text(img, text, point, angle, text_scale, text_color, text_thickness):
    img_filled = np.full(img.shape, text_color, dtype=np.uint8)
    # create rotated text mask
    text_mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    cv2.putText(text_mask, "{:.2f} cm".format(text), point, 0, text_scale, (255, 255, 255), text_thickness)
    if angle > 0:
        angle = -angle + 90
    elif angle < 0:
        angle = angle + 90
    text_mask = rotate_image(text_mask, -angle, point)
    result = copy_to(img_filled, img.copy(), text_mask)
    return result

def draw_real_sizes(img, rect, width_text, height_text, lbl_size_scale=2, lbl_color=(0, 0, 255), lbl_thickness=8):
    tl, tr, br, bl = order_points(cv2.boxPoints(rect))
    mid_pt_width = midpoint(tl, tr)
    mid_pt_height = midpoint(tr, br)
    
    # bottom-left points where labels are drawn
    pt_label_first =  (int(mid_pt_width[0] - 10), int(mid_pt_width[1] - 10))
    pt_label_second = (int(mid_pt_height[0] + 10), int(mid_pt_height[1]))
        
    result = draw_rotated_text(img, width_text, pt_label_first, rect[2], lbl_size_scale, lbl_color, lbl_thickness)
    result = draw_rotated_text(result, height_text, pt_label_second, rect[2], lbl_size_scale, lbl_color, lbl_thickness)
    return result

---
### Kalibrace kamery

<a id='load_save_functions'>Funkce sloužící k načítaní/ukládání kalibrace ze souboru.</a>

In [ ]:
IDX_CAM_MATRIX = "camera_matrix"
IDX_DIST_COEFFS = "dist_coefs"

def load_camera_calib(input_file):
    """ Loads camera calibration from specified input file.
    
    Parameters
    ----------
    input_file : string
        Input file with calibration data in YAML format.
    Returns
    -------
    tuple(np.array, np.array)
        Returns a tuple where first element is camera matrix array and second element is dist coefficients array. 
        These arrays might be empty if the file isn't found or in correct format.
    """
    try:
        with open(input_file, 'r') as stream:
            data = yaml.load(stream)
            return data[IDX_CAM_MATRIX], data[IDX_DIST_COEFFS]
    except (FileNotFoundError, yaml.YAMLError) as exc:
        print(f'File {input_file} couldn\'t be read.')
        return np.array([]), np.array([])

def save_camera_calib(output_file, camera_matrix, dist_coefs):
    """ Saves camera calibration to specified output file.
    
    Parameters
    ----------
    output_file : string
        Output file used for storing calibration data in YAML format. Parent directory is created if needed.
    Returns
    -------
    None
    """
    data = {IDX_CAM_MATRIX: camera_matrix, IDX_DIST_COEFFS: dist_coefs}
    output_dir = os.path.dirname(output_file)
    
    if not os.path.isdir(output_dir):
            os.mkdir(output_dir)
            
    with open(output_file, "w") as f:
        yaml.dump(data, f)

<a id='correct_functions'>Funkce k opravě obrázku pomocí získaných kalibračních dat.</a>

In [ ]:
def correct_frame(frame, camera_matrix, dist_coeffs):
    """Returns undistorted frame."""
    return cv2.undistort(frame, camera_matrix, dist_coeffs)

<a id='reindex_functions'>Funkce sloužící k přejmenování souborů ve složce</a>, aby je bylo možné snadně použít v cv2.VideoCapture.

In [ ]:
def reindex_image_files(source_dir, output_dir=None):
    """ Reads all images in source_dir and based on they original order, 
    change their filename to be continuous integer (starting from 0). 
    Then, they can be easily read by cv2.VideoCapture. Image format is kept.
    
    Parameters
    ----------
    source_dir : string
        Input images directory that have to be renamed.
    output_dir : Optional[string]
        Output directory for renamed files. If not specified, renaming is done inplace in source_dir.
    Returns
    -------
    None
    """
    input_files = []
    
    for file in os.listdir(source_dir):
        if re.match(r'.*(\.bmp|\.jpg|\.png|\.gif)$', file, re.I):
            input_files.append(os.path.join(source_dir, file))

    if not input_files:
        print('No files were found.')
        return
    
    extension = '.' + input_files[0].split(".")[-1]
    if output_dir is None:
        for i, filename in enumerate(natsorted(input_files)):
            os.rename(filename, os.path.join(source_dir, str(i) +  extension))
        print(f'Files within {source_dir} were renamed, starting from 0{extension} to {i}{extension}.')
    else:
        if not os.path.isdir(output_dir):
            os.mkdir(output_dir)

        for i, filename in enumerate(natsorted(input_files)):
            shutil.copy(filename, os.path.join(output_dir, str(i) + extension))
            
        print(f'Files from {source_dir} were renamed and saved to {output_dir}, starting from 0{extension} to {i}{extension}.')

<a id='pick_functions'>Funkce k rychlému výběru chtěných snímků z obrazového zdroje.</a>

In [ ]:
def pick_frames(input_source, output_dir, wait_time):
    """ Sequentially shows all images from input_source. 
    Using 'p' key allows to pick wanted frames and save them in separate folder (output_dir).
    The preview is terminated by pressing the 'q' key.
    
    Parameters
    ----------
    input_source : string
        Input source for cv2.VideoCapture (could be camera source or sequence of saved images where format has to be specified).
    output_dir : string
        Output directory for picked files. Automatically created if needed.
    wait_time : int
        Delay in ms between shown images.
    Returns
    -------
    None
    """
    window_name = 'Frame preview'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, CAM_WIDTH, CAM_HEIGHT)
    cv2.moveWindow(window_name, 0, 0)
    
    cap = cv2.VideoCapture(input_source)
    
    if not cap.isOpened():
        cv2.destroyAllWindows()
        raise FileNotFoundError('Capture cannot be opened.')
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    
    saved_counter = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: 
            break
            
        cv2.imshow(window_name, frame)
        k = cv2.waitKey(wait_time) & 0xFF
    
        if k == ord('p'):
            img_format = input_source.split('.')[-1] if '.' in input_source else 'jpg'
            cv2.imwrite(os.path.join(output_dir, str(saved_counter) + '.' + img_format), frame)
            saved_counter += 1
        elif k == ord('q'):
            break

    cv2.destroyAllWindows()
    print(f'{saved_counter} frames saved to {output_dir}')          

<a id='calib_functions'>Funkce provádějící kalibraci kamery</a>, kterou je možné uložit k pozdějšímu využití.

In [ ]:
def camera_calib(input_source, chess_shape, output_calib_file=None, img_show_delay=1):
    """ Browses all images found in input_source and on each image tries to find chessboard corners.
    If chessboard corners are found, image corespondences with real world space are added to lists.
    Based on these image-world corespondences, camera calibration is made.
    
    Parameters
    ----------
    input_source : string
        Input source for cv2.VideoCapture (could be camera source or sequence of saved images where format has to be specified).
    chess_shape : tuple
        Number of inner corners per a chessboard row and column.
    output_calib_file : Optional[string]
        Output file where calibration is saved when neccesary.
    img_show_delay : int
        Delay in ms between shown images.
    Returns
    -------
    tuple
        camera matrix and distance coefficients
    """
    window_name = 'Frame preview'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, CAM_WIDTH, CAM_HEIGHT)
    cv2.moveWindow(window_name, 0, 0)

    cap = cv2.VideoCapture(input_source)

    if not cap.isOpened():
        cv2.destroyAllWindows()
        raise FileNotFoundError('Capture cannot be opened.')

    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((chess_shape[0] * chess_shape[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:chess_shape[0], 0:chess_shape[1]].T.reshape(-1, 2)

    # Arrays to store object points and image points from all the images.
    objpoints = []  # 3d point in real world space
    imgpoints = []  # 2d points in image plane.

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, chess_shape)

        # If found, add object points, image points (after refining them)
        if ret:
            objpoints.append(objp)
            corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1))
            imgpoints.append(corners)

            # Draw and display the corners
            frame = cv2.drawChessboardCorners(frame, chess_shape, corners, ret)

        cv2.imshow(window_name, frame)

        k = cv2.waitKey(img_show_delay) & 0xFF
        if k == ord('q'):
            break

    cv2.destroyAllWindows()
    
    print('Computing camera matrix...')
    rms, camera_matrix, dist_coefs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
    if output_calib_file is not None:
        save_camera_calib(output_calib_file, camera_matrix, dist_coefs)

    print('\nRMS:', rms)
    print('Camera matrix:\n', camera_matrix)
    print('Distortion coefficients: ', dist_coefs.ravel())
    
    return camera_matrix, dist_coefs

<a id='path_functions'>Funkce pro vytvoření cesty z názvů složek a názvů souboru.</a>

In [2]:
def create_folder_path(base_folder, new_folder_name):
    """ Creates all neccessary folders in the folder tree structure on computer. 
    
    Parameters
    ----------
    base_folder : string
        Base folder directory in string notation. 
    output_dir : string
        Folder name that should be inside the base folder.
    Returns
    -------
    string
        Path to the newly created folder.
    """
    if not os.path.isdir(base_folder):
        os.mkdir(base_folder)
            
    path = os.path.join(base_folder, new_folder_name)        
    
    if not os.path.isdir(path):
        os.mkdir(path)      
            
    return path

def create_file_path(folder, file_name):
    '''Easier defined function to create path for filename inside a folder.
    
    Parameters
    ----------
    folder : string
        Base folder directory in string notation. 
    file_name : string
        File name that should be inside the base folder.
    Returns
    -------
    string
        Path to the newly created file.
    """
    '''
    if not os.path.isdir(folder):
        os.mkdir(folder)
        
    return os.path.join(folder, file_name)

---
### Odhad výšky
<a id='height_estimation_functions'>Hotové funkce a modely pro výpočty.</a>

In [ ]:
# Funkce pro konverze mezi typy souřadnic
def convert_pt_to_homogenous(pt):
    """
    Convert input point in inhomogeneous coordinates to homogeneous.

    :param pt: ndarray
        Input point in inhomogeneous coordinates.
    :return: ndarray
        Input point in homogeneous coordinates.
    """
    return np.append(pt, np.array(1))

def convert_pt_from_homogenous(pt):
    """
    Convert input point in homogeneous coordinates to inhomogeneous.

    :param pt: ndarray
        Input point in homogeneous coordinates.
    :return: ndarray
        Input point in inhomogeneous coordinates.
    """
    return tuple([elem / pt[-1] for elem in pt[:-1]])

def convert_pts_to_homogenous(pts):
    """
    Convert input points in inhomogeneous coordinates to homogeneous.

    :param pt: ndarray
        Input points in inhomogeneous coordinates.
    :return: ndarray
        Input points in homogeneous coordinates.
    """
    return np.array([convert_pt_to_homogenous(pt) for pt in pts])

def convert_pts_from_homogenous(pts):
    """
    Convert input points in homogeneous coordinates to inhomogeneous.

    :param pt: ndarray
        Input points in homogeneous coordinates.
    :return: ndarray
        Input points in inhomogeneous coordinates.
    """
    return np.array([convert_pt_from_homogenous(pt) for pt in pts])

def _calc_alfa_metric_factor(ref_measurements, vanish_line, vert_vanish_point):
    """ Calculates alfa metric factor using multiple reference measurements via minimization ||As|| = 0. This is done by SVD.
        In depth overview can be found in https://www.robots.ox.ac.uk/~vgg/publications/1999/Criminisi99b/criminisi99b.pdf - PDF page 104.

    :param ref_measurements: list
        Each measurement is in (t_ref, b_ref, height) format. Image coordinates are in inhomogeneous format.
    :param vanish_line: ndarray
        Homogenous coordinates of vanishing line.
    :param vert_vanish_point: ndarray
        Homogenous coordinates of vanishing point in reference direction.
    :return: float
        Scalar value of alfa metric factor calculated by SVD.
    """
    matrix_A = np.empty((len(ref_measurements), 2), dtype='float64')

    for i, (t_ref, b_ref, h_ref) in enumerate(ref_measurements):
        t_ref = convert_pt_to_homogenous(t_ref)
        b_ref = convert_pt_to_homogenous(b_ref)
        beta = np.linalg.norm(np.cross(b_ref, t_ref))
        ro = np.dot(vanish_line, b_ref)
        gamma = np.linalg.norm(np.cross(vert_vanish_point, t_ref))
        matrix_A[i] = (h_ref * ro * gamma, beta)
        # alfa_metric_factor = - np.linalg.norm(np.cross(b_ref, t_ref)) / \
        #                               (h_ref * (np.dot(vanish_line, b_ref)) * np.linalg.norm(np.cross(vert_vanish_point, t_ref)))
        # print(alfa_metric_factor)

    u, s, vh = np.linalg.svd(matrix_A)
    return vh[0, -1] / vh[1, -1]

In [ ]:
# Třída pro počítání odhadu výšky
class HeightEstimator:
    
    def __init__(self, ref_measurements, vl, vz):
        self._vanish_line = vl
        self._vert_vanish_point = vz
        self._alfa_metric_factor = _calc_alfa_metric_factor(ref_measurements, self._vanish_line, self._vert_vanish_point)

    def calc_height(self, top_point, bottom_point):
        """
        Calculates real world height based on top_point and bottom_point measured on image plane.

        :param top_point: ndarray
            Top point in reference direction of the object in inhomogeneous format.
        :param bottom_point: ndarray
            Ground plane point of the object in inhomogeneous format.
        :return: float
            Scalar value representing real world height.
        """
        top_point = convert_pt_to_homogenous(top_point)
        bottom_point = convert_pt_to_homogenous(bottom_point)
        # This formula comes from paper Single view metrology by A. Criminisi.
        height = - np.linalg.norm(np.cross(bottom_point, top_point)) / (
                self._alfa_metric_factor * (np.dot(self._vanish_line, bottom_point)) * np.linalg.norm(np.cross(self._vert_vanish_point, top_point)))
        return height

In [ ]:
# Algoritmus automatizovaného výpočtu úběžníků a úběžnic v obraze
"""
References
----------
1.  Chaudhury, Krishnendu, Stephen DiVerdi, and Sergey Ioffe.
    "Auto-rectification of user photos." 2014 IEEE International Conference on
    Image Processing (ICIP). IEEE, 2014.
2.  Bazin, Jean-Charles, and Marc Pollefeys. "3-line RANSAC for orthogonal
    vanishing point detection." 2012 IEEE/RSJ International Conference on
    Intelligent Robots and Systems. IEEE, 2012.
"""
def compute_edgelets(image, sigma=3):
    """Create edgelets as in the paper.

    Uses canny edge detection and then finds (small) lines using probabilstic
    hough transform as edgelets.

    Parameters
    ----------
    image: ndarray
        Image for which edgelets are to be computed.
    sigma: float
        Smoothing to be used for canny edge detection.

    Returns
    -------
    locations: ndarray of shape (n_edgelets, 2)
        Locations of each of the edgelets.
    directions: ndarray of shape (n_edgelets, 2)
        Direction of the edge (tangent) at each of the edgelet.
    strengths: ndarray of shape (n_edgelets,)
        Length of the line segments detected for the edgelet.
    """
    gray_img = color.rgb2gray(image)
    edges = feature.canny(gray_img, sigma)
    lines = transform.probabilistic_hough_line(edges, line_length=3,
                                               line_gap=2)

    locations = []
    directions = []
    strengths = []

    for p0, p1 in lines:
        p0, p1 = np.array(p0), np.array(p1)
        locations.append((p0 + p1) / 2)
        directions.append(p1 - p0)
        strengths.append(np.linalg.norm(p1 - p0))

    # convert to numpy arrays and normalize
    locations = np.array(locations)
    directions = np.array(directions)
    strengths = np.array(strengths)

    directions = np.array(directions) / \
        np.linalg.norm(directions, axis=1)[:, np.newaxis]

    return (locations, directions, strengths)


def edgelet_lines(edgelets):
    """Compute lines in homogenous system for edglets.

    Parameters
    ----------
    edgelets: tuple of ndarrays
        (locations, directions, strengths) as computed by `compute_edgelets`.

    Returns
    -------
    lines: ndarray of shape (n_edgelets, 3)
        Lines at each of edgelet locations in homogenous system.
    """
    locations, directions, _ = edgelets
    normals = np.zeros_like(directions)
    normals[:, 0] = directions[:, 1]
    normals[:, 1] = -directions[:, 0]
    p = -np.sum(locations * normals, axis=1)
    lines = np.concatenate((normals, p[:, np.newaxis]), axis=1)
    return lines


def compute_votes(edgelets, model, threshold_inlier=5):
    """Compute votes for each of the edgelet against a given vanishing point.

    Votes for edgelets which lie inside threshold are same as their strengths,
    otherwise zero.

    Parameters
    ----------
    edgelets: tuple of ndarrays
        (locations, directions, strengths) as computed by `compute_edgelets`.
    model: ndarray of shape (3,)
        Vanishing point model in homogenous cordinate system.
    threshold_inlier: float
        Threshold to be used for computing inliers in degrees. Angle between
        edgelet direction and line connecting the  Vanishing point model and
        edgelet location is used to threshold.

    Returns
    -------
    votes: ndarry of shape (n_edgelets,)
        Votes towards vanishing point model for each of the edgelet.

    """
    vp = model[:2] / model[2]

    locations, directions, strengths = edgelets

    est_directions = locations - vp
    dot_prod = np.sum(est_directions * directions, axis=1)
    abs_prod = np.linalg.norm(directions, axis=1) * \
        np.linalg.norm(est_directions, axis=1)
    abs_prod[abs_prod == 0] = 1e-5

    cosine_theta = dot_prod / abs_prod
    theta = np.arccos(np.abs(cosine_theta))

    theta_thresh = threshold_inlier * np.pi / 180
    return (theta < theta_thresh) * strengths


def ransac_vanishing_point(edgelets, num_ransac_iter=2000, threshold_inlier=5):
    """Estimate vanishing point using Ransac.

    Parameters
    ----------
    edgelets: tuple of ndarrays
        (locations, directions, strengths) as computed by `compute_edgelets`.
    num_ransac_iter: int
        Number of iterations to run ransac.
    threshold_inlier: float
        threshold to be used for computing inliers in degrees.

    Returns
    -------
    best_model: ndarry of shape (3,)
        Best model for vanishing point estimated.

    Reference
    ---------
    Chaudhury, Krishnendu, Stephen DiVerdi, and Sergey Ioffe.
    "Auto-rectification of user photos." 2014 IEEE International Conference on
    Image Processing (ICIP). IEEE, 2014.
    """
    locations, directions, strengths = edgelets
    lines = edgelet_lines(edgelets)

    num_pts = strengths.size

    arg_sort = np.argsort(-strengths)
    first_index_space = arg_sort[:num_pts // 5]
    second_index_space = arg_sort[:num_pts // 2]

    best_model = None
    best_votes = np.zeros(num_pts)

    for ransac_iter in range(num_ransac_iter):
        ind1 = np.random.choice(first_index_space)
        ind2 = np.random.choice(second_index_space)

        l1 = lines[ind1]
        l2 = lines[ind2]

        current_model = np.cross(l1, l2)

        if np.sum(current_model**2) < 1 or current_model[2] == 0:
            # reject degenerate candidates
            continue

        current_votes = compute_votes(
            edgelets, current_model, threshold_inlier)

        if current_votes.sum() > best_votes.sum():
            best_model = current_model
            best_votes = current_votes

    return best_model


def reestimate_model(model, edgelets, threshold_reestimate=5):
    """Reestimate vanishing point using inliers and least squares.

    All the edgelets which are within a threshold are used to reestimate model

    Parameters
    ----------
    model: ndarry of shape (3,)
        Vanishing point model in homogenous coordinates which is to be
        reestimated.
    edgelets: tuple of ndarrays
        (locations, directions, strengths) as computed by `compute_edgelets`.
        All edgelets from which inliers will be computed.
    threshold_inlier: float
        threshold to be used for finding inlier edgelets.

    Returns
    -------
    restimated_model: ndarry of shape (3,)
        Reestimated model for vanishing point in homogenous coordinates.
    """
    locations, directions, strengths = edgelets

    inliers = compute_votes(edgelets, model, threshold_reestimate) > 0
    locations = locations[inliers]
    directions = directions[inliers]
    strengths = strengths[inliers]

    lines = edgelet_lines((locations, directions, strengths))

    a = lines[:, :2]
    b = -lines[:, 2]
    est_model = np.linalg.lstsq(a, b)[0]
    return np.concatenate((est_model, [1.]))


def remove_inliers(model, edgelets, threshold_inlier=10):
    """Remove all inlier edglets of a given model.

    Parameters
    ----------
    model: ndarry of shape (3,)
        Vanishing point model in homogenous coordinates which is to be
        reestimated.
    edgelets: tuple of ndarrays
        (locations, directions, strengths) as computed by `compute_edgelets`.
    threshold_inlier: float
        threshold to be used for finding inlier edgelets.

    Returns
    -------
    edgelets_new: tuple of ndarrays
        All Edgelets except those which are inliers to model.
    """
    inliers = compute_votes(edgelets, model, 10) > 0
    locations, directions, strengths = edgelets
    locations = locations[~inliers]
    directions = directions[~inliers]
    strengths = strengths[~inliers]
    edgelets = (locations, directions, strengths)
    return edgelets


def compute_homography_and_warp(image, vp1, vp2, clip=True, clip_factor=3):
    """Compute homography from vanishing points and warp the image.

    It is assumed that vp1 and vp2 correspond to horizontal and vertical
    directions, although the order is not assumed.
    Firstly, projective transform is computed to make the vanishing points go
    to infinty so that we have a fronto parellel view. Then,Computes affine
    transfom  to make axes corresponding to vanishing points orthogonal.
    Finally, Image is translated so that the image is not missed. Note that
    this image can be very large. `clip` is provided to deal with this.

    Parameters
    ----------
    image: ndarray
        Image which has to be wrapped.
    vp1: ndarray of shape (3, )
        First vanishing point in homogenous coordinate system.
    vp2: ndarray of shape (3, )
        Second vanishing point in homogenous coordinate system.
    clip: bool, optional
        If True, image is clipped to clip_factor.
    clip_factor: float, optional
        Proportion of image in multiples of image size to be retained if gone
        out of bounds after homography.
    Returns
    -------
    warped_img: ndarray
        Image warped using homography as described above.
    """
    # Find Projective Transform
    vanishing_line = np.cross(vp1, vp2)
    H = np.eye(3)
    H[2] = vanishing_line / vanishing_line[2]
    H = H / H[2, 2]

    # Find directions corresponding to vanishing points
    v_post1 = np.dot(H, vp1)
    v_post2 = np.dot(H, vp2)
    v_post1 = v_post1 / np.sqrt(v_post1[0]**2 + v_post1[1]**2)
    v_post2 = v_post2 / np.sqrt(v_post2[0]**2 + v_post2[1]**2)

    directions = np.array([[v_post1[0], -v_post1[0], v_post2[0], -v_post2[0]],
                           [v_post1[1], -v_post1[1], v_post2[1], -v_post2[1]]])

    thetas = np.arctan2(directions[0], directions[1])

    # Find direction closest to horizontal axis
    h_ind = np.argmin(np.abs(thetas))

    # Find positve angle among the rest for the vertical axis
    if h_ind // 2 == 0:
        v_ind = 2 + np.argmax([thetas[2], thetas[3]])
    else:
        v_ind = np.argmax([thetas[2], thetas[3]])

    A1 = np.array([[directions[0, v_ind], directions[0, h_ind], 0],
                   [directions[1, v_ind], directions[1, h_ind], 0],
                   [0, 0, 1]])
    # Might be a reflection. If so, remove reflection.
    if np.linalg.det(A1) < 0:
        A1[:, 0] = -A1[:, 0]

    A = np.linalg.inv(A1)

    # Translate so that whole of the image is covered
    inter_matrix = np.dot(A, H)

    cords = np.dot(inter_matrix, [[0, 0, image.shape[1], image.shape[1]],
                                  [0, image.shape[0], 0, image.shape[0]],
                                  [1, 1, 1, 1]])
    cords = cords[:2] / cords[2]

    tx = min(0, cords[0].min())
    ty = min(0, cords[1].min())

    max_x = cords[0].max() - tx
    max_y = cords[1].max() - ty

    if clip:
        # These might be too large. Clip them.
        max_offset = max(image.shape) * clip_factor / 2
        tx = max(tx, -max_offset)
        ty = max(ty, -max_offset)

        max_x = min(max_x, -tx + max_offset)
        max_y = min(max_y, -ty + max_offset)

    max_x = int(max_x)
    max_y = int(max_y)

    T = np.array([[1, 0, -tx],
                  [0, 1, -ty],
                  [0, 0, 1]])

    final_homography = np.dot(T, inter_matrix)

    warped_img = transform.warp(image, np.linalg.inv(final_homography),
                                output_shape=(max_y, max_x))
    return warped_img


def vis_edgelets(image, edgelets, show=True):
    """Helper function to visualize edgelets."""
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    locations, directions, strengths = edgelets
    for i in range(locations.shape[0]):
        xax = [locations[i, 0] - directions[i, 0] * strengths[i] / 2,
               locations[i, 0] + directions[i, 0] * strengths[i] / 2]
        yax = [locations[i, 1] - directions[i, 1] * strengths[i] / 2,
               locations[i, 1] + directions[i, 1] * strengths[i] / 2]

        plt.plot(xax, yax, 'r-')

    if show:
        plt.show()


def vis_model(image, model, show=True):
    """Helper function to visualize computed model."""
    edgelets = compute_edgelets(image)
    locations, directions, strengths = edgelets
    inliers = compute_votes(edgelets, model, 10) > 0

    edgelets = (locations[inliers], directions[inliers], strengths[inliers])
    locations, directions, strengths = edgelets
    vis_edgelets(image, edgelets, False)
    vp = model / model[2]
    plt.plot(vp[0], vp[1], 'bo')
    for i in range(locations.shape[0]):
        xax = [locations[i, 0], vp[0]]
        yax = [locations[i, 1], vp[1]]
        plt.plot(xax, yax, 'b-.')

    if show:
        plt.show()


def compute_vanishing_points(image, clip_factor=6, reestimate=False):
    """Rectified image with vanishing point computed using ransac.

    Parameters
    ----------
    image: ndarray
        Image which has to be rectified.
    clip_factor: float, optional
        Proportion of image in multiples of image size to be retained if gone
        out of bounds after homography.
    reestimate: bool
        If ransac results are to be reestimated using least squares with
        inlers. Turn this off if getting bad results.
    Returns
    -------
    warped_img: ndarray
        Rectified image.
    """
    #     if type(image) is not np.ndarray:
    #         image = io.imread(image)

    # Compute all edgelets.
    edgelets1 = compute_edgelets(image)
    
    vps = []

    # Find first vanishing point
    vp1 = ransac_vanishing_point(edgelets1, 2000, threshold_inlier=5)
    if reestimate:
        vp1 = reestimate_model(vp1, edgelets1, 5)

    vps.append(vp1)
    
    
    # Remove inlier to remove dominating direction.
    edgelets2 = remove_inliers(vp1, edgelets1, 10)

    # Find second vanishing point
    vp2 = ransac_vanishing_point(edgelets2, 2000, threshold_inlier=5)
    if reestimate:
        vp2 = reestimate_model(vp2, edgelets2, 5)
    vps.append(vp2)

    edgelets3 = remove_inliers(vp2, edgelets2, 10)

    # Find third vanishing point
    vp3 = ransac_vanishing_point(edgelets3, 2000, threshold_inlier=5)
    if reestimate:
        vp3 = reestimate_model(vp3, edgelets3, 5)
    vps.append(vp3)

    # Compute the homography and warp
    #     warped_img = compute_homography_and_warp(image, vp1, vp2, clip_factor=clip_factor)
    
    # Print results
    for i, vp in enumerate(vps):
        print(f'vp{i+1} = [{vp[0]}, {vp[1]}, {vp[2]}]')
        vis_model(image, vp)
    
    return vps